In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
n_inputs = 28
max_time = 28
lstm_size = 100 #隐层单元
n_classes = 10
batch_size = 50
n_batch = mnist.train.num_examples // batch_size

In [4]:
def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

In [5]:
x = tf.placeholder(tf.float32, [None,784])
y = tf.placeholder(tf.float32, [None,10])

with tf.name_scope('weights'):
    weights = tf.Variable(tf.truncated_normal([lstm_size,n_classes],stddev=0.1))
    variable_summaries(weights)

with tf.name_scope('biases'):
    biases = tf.Variable(tf.constant(0.1,shape=[n_classes]))
    variable_summaries(biases)

def RNN(X,weights,biases):
    with tf.name_scope('RNN'):
        inputs = tf.reshape(X,[-1,max_time,n_inputs])
        lstm_cell = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        outputs, final_state = tf.nn.dynamic_rnn(lstm_cell,inputs,dtype=tf.float32)
        results = tf.nn.softmax(tf.matmul(final_state[1],weights)+biases)
        tf.summary.histogram('RNN_softmax', results)
        return results
    
prediction = RNN(x,weights,biases)

# loss
with tf.name_scope('cross_entropy'):
    diff = tf.nn.softmax_cross_entropy_with_logits(logits=prediction,labels=y)
    with tf.name_scope('total'):
        loss = tf.reduce_mean(diff)
tf.summary.scalar('cross_entropy', loss)

# optimizer
with tf.name_scope('train'):
    optimizer = tf.train.AdamOptimizer(1e-4)
    train_step = optimizer.minimize(loss)

# 结果
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


In [10]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    # Merge all the summaries and write them out to /tmp/mnist_logs (by default)
    merged = tf.summary.merge_all()
    train_writer = tf.summary.FileWriter('./logs/train',sess.graph)
    test_writer = tf.summary.FileWriter('./logs/test')
    sess.run(init)
    for epoch in range(6):
        for batch in range(n_batch):
            batch_xs,batch_ys = mnist.train.next_batch(batch_size)
            summary,_ = sess.run([merged,train_step],feed_dict={x:batch_xs,y:batch_ys})
            train_writer.add_summary(summary, batch)
        summary,acc = sess.run([merged,accuracy],feed_dict={x:mnist.test.images,y:mnist.test.labels})
        test_writer.add_summary(summary, epoch)
        print("Test step:",epoch, ", Test accuracy:",acc)
    train_writer.close()
    test_writer.close()

Test step: 0 , Test accuracy: 0.6778
Test step: 1 , Test accuracy: 0.8677
Test step: 2 , Test accuracy: 0.9058
Test step: 3 , Test accuracy: 0.918
Test step: 4 , Test accuracy: 0.9245
Test step: 5 , Test accuracy: 0.9348
